In [1]:
import wave
import numpy

In [2]:
# extracts everything (hopefully) relevant from the wav file
def extract_data(wav_file):
    try:
        with wave.open(wav_file, 'rb') as input:
            frames = input.readframes(-1)
            rate = input.getframerate()
            channels = input.getnchannels()
            width = input.getsampwidth()
        return frames, rate, channels, width
    except FileNotFoundError:
        print(f"Error: File '{input_file}' not found.")
        return None
    except IOError as e:
        print(f"Error: Unable to open file '{input_file}': {e}")
        return None

In [3]:
# samp
extracted_data = extract_data('sample.wav')
if extracted_data is None:
    print("Error: no data")
else:
    frames, rate, channels, width = extracted_data
    print("# Frames:", len(frames))
    print("Rate:", rate)
    print("Channels:", channels)
    print("Width:", width)

# Frames: 19840702
Rate: 11025
Channels: 1
Width: 2
